In [1]:
!nvidia-smi

Mon Jan  5 10:39:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Часть 1. Реализация параллельного стека на CUDA**

**Задание:** Реализовать структуру данных стек с использованием атомарных
операций для безопасного доступа к данным.

**Задачи:**
1. Инициализировать стек с фиксированной емкостью.
2. Написать ядро CUDA, использующее push и pop параллельно из
нескольких потоков.
3. Проверить корректность выполнения операций.


In [6]:
%%writefile stack_atomic.cu
#include <cuda_runtime.h>                     // подключаем базовые функции CUDA
#include <cstdio>                             // подключаем printf
#include <vector>                             // подключаем vector для проверки на CPU

struct Stack {                                // объявляем структуру стека
    int *data;                                // указатель на буфер данных в памяти GPU
    int top;                                  // индекс вершины стека
    int capacity;                             // максимальная емкость стека

    __device__ void init(int *buffer, int size) { // метод инициализации стека на устройстве
        data = buffer;                        // сохраняем адрес буфера
        top = -1;                             // делаем стек пустым
        capacity = size;                      // сохраняем емкость
    }

    __device__ bool push(int value) {         // метод добавления элемента в стек
        int pos = atomicAdd(&top, 1) + 1;     // атомарно увеличиваем top и получаем позицию записи
        if (pos < capacity) {                 // проверяем что позиция не вышла за емкость
            data[pos] = value;                // записываем значение в стек
            return true;                      // возвращаем успех
        }
        atomicSub(&top, 1);                   // откатываем top если произошел выход за емкость
        return false;                         // возвращаем неуспех
    }

    __device__ bool pop(int *value) {         // метод извлечения элемента из стека
        int pos = atomicSub(&top, 1);         // атомарно уменьшаем top и получаем позицию чтения
        if (pos >= 0) {                       // проверяем что стек не пуст
            *value = data[pos];               // читаем значение из стека
            return true;                      // возвращаем успех
        }
        atomicAdd(&top, 1);                   // откатываем top если стек был пуст
        return false;                         // возвращаем неуспех
    }
};

__global__ void init_stack(Stack *s, int *buffer, int capacity) { // ядро инициализации стека
    if (threadIdx.x == 0 && blockIdx.x == 0) {                    // выполняем только одним потоком
        s->init(buffer, capacity);                                // вызываем init на устройстве
    }
}

__global__ void push_kernel(Stack *s, int *push_ok, int n) {      // ядро параллельного push
    int tid = blockIdx.x * blockDim.x + threadIdx.x;              // считаем глобальный индекс потока
    if (tid < n) {                                                // проверяем границу
        bool ok = s->push(tid);                                   // кладем tid в стек
        push_ok[tid] = ok ? 1 : 0;                                // сохраняем результат операции
    }
}

__global__ void pop_kernel(Stack *s, int *out, int *pop_ok, int n) { // ядро параллельного pop
    int tid = blockIdx.x * blockDim.x + threadIdx.x;                 // считаем глобальный индекс потока
    if (tid < n) {                                                   // проверяем границу
        int val = -1;                                                // локальная переменная для результата
        bool ok = s->pop(&val);                                      // пытаемся извлечь значение
        out[tid] = val;                                              // сохраняем извлеченное значение
        pop_ok[tid] = ok ? 1 : 0;                                    // сохраняем результат операции
    }
}

static void checkCuda(cudaError_t err, const char *msg) {         // функция проверки ошибок CUDA
    if (err != cudaSuccess) {                                     // если есть ошибка
        std::printf("CUDA error: %s : %s\n", msg, cudaGetErrorString(err)); // печатаем текст ошибки
        std::exit(1);                                             // завершаем программу
    }
}

int main() {                                                      // точка входа программы
    const int N = 100000;                                         // количество потоков для теста
    const int CAPACITY = N;                                       // емкость стека

    Stack *d_stack = nullptr;                                     // указатель на стек в managed памяти
    int *d_data = nullptr;                                        // указатель на буфер стека в managed памяти
    int *d_push_ok = nullptr;                                     // массив флагов успешного push
    int *d_pop_ok = nullptr;                                      // массив флагов успешного pop
    int *d_out = nullptr;                                         // массив извлеченных значений

    checkCuda(cudaMallocManaged(&d_stack, sizeof(Stack)), "malloc stack");      // выделяем память под структуру стека
    checkCuda(cudaMallocManaged(&d_data, CAPACITY * sizeof(int)), "malloc data"); // выделяем память под данные стека
    checkCuda(cudaMallocManaged(&d_push_ok, N * sizeof(int)), "malloc push_ok"); // выделяем память под флаги push
    checkCuda(cudaMallocManaged(&d_pop_ok, N * sizeof(int)), "malloc pop_ok");   // выделяем память под флаги pop
    checkCuda(cudaMallocManaged(&d_out, N * sizeof(int)), "malloc out");         // выделяем память под выход

    init_stack<<<1, 1>>>(d_stack, d_data, CAPACITY);              // запускаем инициализацию стека
    checkCuda(cudaGetLastError(), "launch init_stack");           // проверяем ошибку запуска
    checkCuda(cudaDeviceSynchronize(), "sync init_stack");        // ждем завершения инициализации

    int threads = 256;                                            // размер блока
    int blocks = (N + threads - 1) / threads;                     // количество блоков

    cudaEvent_t e1, e2;                                           // события для измерения времени
    checkCuda(cudaEventCreate(&e1), "event create e1");           // создаем событие начала
    checkCuda(cudaEventCreate(&e2), "event create e2");           // создаем событие конца

    checkCuda(cudaEventRecord(e1), "event record start push");    // ставим начало измерения push
    push_kernel<<<blocks, threads>>>(d_stack, d_push_ok, N);      // запускаем параллельный push
    checkCuda(cudaGetLastError(), "launch push_kernel");          // проверяем ошибку запуска push
    checkCuda(cudaDeviceSynchronize(), "sync push_kernel");       // ждем завершения push
    checkCuda(cudaEventRecord(e2), "event record end push");      // ставим конец измерения push
    checkCuda(cudaEventSynchronize(e2), "event sync end push");   // синхронизируем событие
    float push_ms = 0.0f;                                         // переменная времени push
    checkCuda(cudaEventElapsedTime(&push_ms, e1, e2), "elapsed push"); // считаем время push

    checkCuda(cudaEventRecord(e1), "event record start pop");     // ставим начало измерения pop
    pop_kernel<<<blocks, threads>>>(d_stack, d_out, d_pop_ok, N); // запускаем параллельный pop
    checkCuda(cudaGetLastError(), "launch pop_kernel");           // проверяем ошибку запуска pop
    checkCuda(cudaDeviceSynchronize(), "sync pop_kernel");        // ждем завершения pop
    checkCuda(cudaEventRecord(e2), "event record end pop");       // ставим конец измерения pop
    checkCuda(cudaEventSynchronize(e2), "event sync end pop");    // синхронизируем событие
    float pop_ms = 0.0f;                                          // переменная времени pop
    checkCuda(cudaEventElapsedTime(&pop_ms, e1, e2), "elapsed pop"); // считаем время pop

    int push_success = 0;                                         // счетчик успешных push
    int pop_success = 0;                                          // счетчик успешных pop

    for (int i = 0; i < N; i++) {                                 // цикл по всем потокам
        push_success += d_push_ok[i];                             // суммируем успехи push
        pop_success += d_pop_ok[i];                               // суммируем успехи pop
    }

    std::vector<char> seen(N, 0);                                 // массив для проверки уникальности значений
    int valid_values = 0;                                         // счетчик корректных извлеченных значений
    int duplicates = 0;                                           // счетчик повторов

    for (int i = 0; i < N; i++) {                                 // цикл проверки извлеченных данных
        if (d_pop_ok[i] == 1) {                                   // если pop был успешным
            int v = d_out[i];                                     // берем значение
            if (v >= 0 && v < N) {                                // проверяем диапазон
                if (seen[v]) duplicates++;                        // если уже было то это повтор
                else {                                            // иначе
                    seen[v] = 1;                                  // отмечаем что значение встречалось
                    valid_values++;                               // увеличиваем счетчик корректных уникальных
                }
            }
        }
    }

    bool ok = (push_success == N) && (pop_success == N) && (valid_values == N) && (duplicates == 0); // итоговая проверка

    std::printf("N: %d\n", N);                                    // печатаем N
    std::printf("Push ms: %.6f\n", push_ms);                      // печатаем время push
    std::printf("Pop  ms: %.6f\n", pop_ms);                       // печатаем время pop
    std::printf("Push success: %d\n", push_success);              // печатаем число успешных push
    std::printf("Pop  success: %d\n", pop_success);               // печатаем число успешных pop
    std::printf("Valid unique values: %d\n", valid_values);       // печатаем число корректных уникальных значений
    std::printf("Duplicates: %d\n", duplicates);                  // печатаем число повторов

    checkCuda(cudaEventDestroy(e1), "event destroy e1");          // удаляем событие
    checkCuda(cudaEventDestroy(e2), "event destroy e2");          // удаляем событие

    checkCuda(cudaFree(d_out), "free out");                       // освобождаем память out
    checkCuda(cudaFree(d_pop_ok), "free pop_ok");                 // освобождаем память pop_ok
    checkCuda(cudaFree(d_push_ok), "free push_ok");               // освобождаем память push_ok
    checkCuda(cudaFree(d_data), "free data");                     // освобождаем память data
    checkCuda(cudaFree(d_stack), "free stack");                   // освобождаем память stack

    return 0;                                                     // завершаем программу
}


Writing stack_atomic.cu


In [7]:
!nvcc -O3 -std=c++17 stack_atomic.cu -o stack_atomic \
  -gencode arch=compute_75,code=sm_75
!./stack_atomic

stack_atomic.cu(138): warning #177-D: variable "ok" was declared but never referenced
      bool ok = (push_success == N) && (pop_success == N) && (valid_values == N) && (duplicates == 0);
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

N: 100000
Push ms: 0.194048
Pop  ms: 0.163808
Push success: 100000
Pop  success: 100000
Valid unique values: 100000
Duplicates: 0


**Часть 2. Реализация параллельной очереди на CUDA**

**Задание:** Реализовать очередь с использованием атомарных операций для
безопасного добавления и удаления элементов.

**Задачи:**
1. Инициализировать очередь с заданной емкостью.
2. Написать ядро CUDA, использующее enqueue и dequeue параллельно.
3. Сравнить производительность очереди и стека.

In [8]:
%%writefile stack_2_atomic.cu
#include <cuda_runtime.h>                                  // подключаем функции CUDA
#include <cstdio>                                          // подключаем printf
#include <vector>                                          // подключаем vector для проверки

static void checkCuda(cudaError_t err, const char *msg) {  // функция проверки ошибок CUDA
    if (err != cudaSuccess) {                              // если ошибка есть
        std::printf("CUDA error: %s : %s\n", msg, cudaGetErrorString(err)); // печатаем ошибку
        std::exit(1);                                      // завершаем программу
    }
}

struct Stack {                                             // структура стека
    int *data;                                             // указатель на буфер данных
    int top;                                               // индекс вершины
    int capacity;                                          // емкость

    __device__ void init(int *buffer, int size) {          // инициализация стека
        data = buffer;                                     // сохраняем буфер
        top = -1;                                          // стек пуст
        capacity = size;                                   // сохраняем емкость
    }

    __device__ bool push(int value) {                      // добавление элемента
        int pos = atomicAdd(&top, 1) + 1;                  // атомарно двигаем top и берем позицию
        if (pos < capacity) {                              // если есть место
            data[pos] = value;                             // записываем значение
            return true;                                   // успех
        }
        atomicSub(&top, 1);                                // откатываем top при переполнении
        return false;                                      // неуспех
    }

    __device__ bool pop(int *value) {                      // извлечение элемента
        int pos = atomicSub(&top, 1);                      // атомарно уменьшаем top и берем позицию
        if (pos >= 0) {                                    // если стек не пуст
            *value = data[pos];                            // читаем значение
            return true;                                   // успех
        }
        atomicAdd(&top, 1);                                // откатываем top при пустом стеке
        return false;                                      // неуспех
    }
};

struct Queue {                                             // структура очереди
    int *data;                                             // указатель на буфер данных
    int head;                                              // индекс чтения
    int tail;                                              // индекс записи
    int capacity;                                          // емкость

    __device__ void init(int *buffer, int size) {          // инициализация очереди
        data = buffer;                                     // сохраняем буфер
        head = 0;                                          // начало очереди
        tail = 0;                                          // конец очереди
        capacity = size;                                   // сохраняем емкость
    }

    __device__ bool enqueue(int value) {                   // добавление в очередь
        int pos = atomicAdd(&tail, 1);                     // атомарно получаем позицию записи
        if (pos < capacity) {                              // если есть место
            data[pos] = value;                             // записываем значение
            return true;                                   // успех
        }
        return false;                                      // неуспех
    }

    __device__ bool dequeue(int *value) {                  // извлечение из очереди
        int pos = atomicAdd(&head, 1);                     // атомарно получаем позицию чтения
        int t = atomicAdd(&tail, 0);                       // атомарно читаем текущий tail
        if (pos < t) {                                     // если элемент уже добавлен
            *value = data[pos];                            // читаем значение
            return true;                                   // успех
        }
        return false;                                      // неуспех
    }
};

__global__ void init_stack(Stack *s, int *buffer, int cap) { // ядро инициализации стека
    if (blockIdx.x == 0 && threadIdx.x == 0) {               // один поток
        s->init(buffer, cap);                                // вызов init
    }
}

__global__ void init_queue(Queue *q, int *buffer, int cap) { // ядро инициализации очереди
    if (blockIdx.x == 0 && threadIdx.x == 0) {               // один поток
        q->init(buffer, cap);                                // вызов init
    }
}

__global__ void stack_push_kernel(Stack *s, int *ok, int n) { // ядро параллельного push
    int tid = blockIdx.x * blockDim.x + threadIdx.x;         // глобальный индекс потока
    if (tid < n) {                                           // проверка границы
        bool r = s->push(tid);                               // кладем tid в стек
        ok[tid] = r ? 1 : 0;                                 // записываем успех
    }
}

__global__ void stack_pop_kernel(Stack *s, int *out, int *ok, int n) { // ядро параллельного pop
    int tid = blockIdx.x * blockDim.x + threadIdx.x;                   // глобальный индекс потока
    if (tid < n) {                                                     // проверка границы
        int v = -1;                                                    // переменная для результата
        bool r = s->pop(&v);                                           // извлекаем из стека
        out[tid] = v;                                                  // сохраняем значение
        ok[tid] = r ? 1 : 0;                                           // записываем успех
    }
}

__global__ void queue_enq_kernel(Queue *q, int *ok, int n) { // ядро параллельного enqueue
    int tid = blockIdx.x * blockDim.x + threadIdx.x;         // глобальный индекс потока
    if (tid < n) {                                           // проверка границы
        bool r = q->enqueue(tid);                            // добавляем tid в очередь
        ok[tid] = r ? 1 : 0;                                 // записываем успех
    }
}

__global__ void queue_deq_kernel(Queue *q, int *out, int *ok, int n) { // ядро параллельного dequeue
    int tid = blockIdx.x * blockDim.x + threadIdx.x;                   // глобальный индекс потока
    if (tid < n) {                                                     // проверка границы
        int v = -1;                                                    // переменная для результата
        bool r = q->dequeue(&v);                                       // извлекаем из очереди
        out[tid] = v;                                                  // сохраняем значение
        ok[tid] = r ? 1 : 0;                                           // записываем успех
    }
}

static bool check_unique_all(const int *out, const int *ok, int n) {   // проверка что извлечены все уникальные значения
    std::vector<char> seen(n, 0);                                      // массив отметок
    int count = 0;                                                     // счетчик уникальных
    for (int i = 0; i < n; i++) {                                      // цикл по результатам
        if (ok[i] == 1) {                                              // если операция успешна
            int v = out[i];                                            // значение
            if (v < 0 || v >= n) return false;                         // проверка диапазона
            if (seen[v]) return false;                                 // повтор значения
            seen[v] = 1;                                               // отмечаем
            count++;                                                   // увеличиваем счетчик
        } else {                                                       // если операция неуспешна
            return false;                                              // для этой лабораторной ожидаем успех у всех
        }
    }
    return count == n;                                                 // должно быть ровно n уникальных
}

static float time_kernel(void (*launcher)(), cudaEvent_t a, cudaEvent_t b) { // заглушка чтобы не использовать
    return 0.0f;                                                           // не используется
}

int main() {                                                               // точка входа
    std::vector<int> Ns = {10000, 100000, 1000000};                        // размеры тестов

    std::printf("N,stack_push_ms,stack_pop_ms,stack_ok,queue_enq_ms,queue_deq_ms,queue_ok\n"); // заголовок CSV

    for (int idx = 0; idx < (int)Ns.size(); idx++) {                       // цикл по размерам
        int N = Ns[idx];                                                   // текущий размер
        int CAP = N;                                                       // емкость под N элементов

        Stack *d_stack = nullptr;                                          // указатель на стек
        Queue *d_queue = nullptr;                                          // указатель на очередь

        int *stack_buf = nullptr;                                          // буфер стека
        int *queue_buf = nullptr;                                          // буфер очереди

        int *s_ok1 = nullptr;                                              // флаги push
        int *s_ok2 = nullptr;                                              // флаги pop
        int *s_out = nullptr;                                              // результаты pop

        int *q_ok1 = nullptr;                                              // флаги enqueue
        int *q_ok2 = nullptr;                                              // флаги dequeue
        int *q_out = nullptr;                                              // результаты dequeue

        checkCuda(cudaMallocManaged(&d_stack, sizeof(Stack)), "malloc stack");         // память под стек
        checkCuda(cudaMallocManaged(&d_queue, sizeof(Queue)), "malloc queue");         // память под очередь
        checkCuda(cudaMallocManaged(&stack_buf, CAP * sizeof(int)), "malloc stack buf"); // буфер стека
        checkCuda(cudaMallocManaged(&queue_buf, CAP * sizeof(int)), "malloc queue buf"); // буфер очереди

        checkCuda(cudaMallocManaged(&s_ok1, N * sizeof(int)), "malloc s_ok1");         // флаги push
        checkCuda(cudaMallocManaged(&s_ok2, N * sizeof(int)), "malloc s_ok2");         // флаги pop
        checkCuda(cudaMallocManaged(&s_out, N * sizeof(int)), "malloc s_out");         // результаты pop

        checkCuda(cudaMallocManaged(&q_ok1, N * sizeof(int)), "malloc q_ok1");         // флаги enqueue
        checkCuda(cudaMallocManaged(&q_ok2, N * sizeof(int)), "malloc q_ok2");         // флаги dequeue
        checkCuda(cudaMallocManaged(&q_out, N * sizeof(int)), "malloc q_out");         // результаты dequeue

        init_stack<<<1, 1>>>(d_stack, stack_buf, CAP);                                // инициализация стека
        checkCuda(cudaGetLastError(), "launch init_stack");                           // проверка запуска
        checkCuda(cudaDeviceSynchronize(), "sync init_stack");                        // синхронизация

        init_queue<<<1, 1>>>(d_queue, queue_buf, CAP);                                // инициализация очереди
        checkCuda(cudaGetLastError(), "launch init_queue");                           // проверка запуска
        checkCuda(cudaDeviceSynchronize(), "sync init_queue");                        // синхронизация

        int threads = 256;                                                            // размер блока
        int blocks = (N + threads - 1) / threads;                                     // число блоков

        cudaEvent_t e1, e2;                                                           // события таймера
        checkCuda(cudaEventCreate(&e1), "event create e1");                           // создаем событие
        checkCuda(cudaEventCreate(&e2), "event create e2");                           // создаем событие

        float stack_push_ms = 0.0f;                                                   // время push
        float stack_pop_ms = 0.0f;                                                    // время pop
        float queue_enq_ms = 0.0f;                                                    // время enqueue
        float queue_deq_ms = 0.0f;                                                    // время dequeue

        checkCuda(cudaEventRecord(e1), "record stack push start");                    // старт замера push
        stack_push_kernel<<<blocks, threads>>>(d_stack, s_ok1, N);                    // запуск push
        checkCuda(cudaGetLastError(), "launch stack_push_kernel");                    // проверка запуска
        checkCuda(cudaDeviceSynchronize(), "sync stack_push_kernel");                 // синхронизация
        checkCuda(cudaEventRecord(e2), "record stack push end");                      // конец замера push
        checkCuda(cudaEventSynchronize(e2), "sync stack push end");                   // синхронизация события
        checkCuda(cudaEventElapsedTime(&stack_push_ms, e1, e2), "elapsed stack push"); // считаем время

        checkCuda(cudaEventRecord(e1), "record stack pop start");                     // старт замера pop
        stack_pop_kernel<<<blocks, threads>>>(d_stack, s_out, s_ok2, N);              // запуск pop
        checkCuda(cudaGetLastError(), "launch stack_pop_kernel");                     // проверка запуска
        checkCuda(cudaDeviceSynchronize(), "sync stack_pop_kernel");                  // синхронизация
        checkCuda(cudaEventRecord(e2), "record stack pop end");                       // конец замера pop
        checkCuda(cudaEventSynchronize(e2), "sync stack pop end");                    // синхронизация события
        checkCuda(cudaEventElapsedTime(&stack_pop_ms, e1, e2), "elapsed stack pop");  // считаем время

        checkCuda(cudaEventRecord(e1), "record queue enq start");                     // старт замера enqueue
        queue_enq_kernel<<<blocks, threads>>>(d_queue, q_ok1, N);                     // запуск enqueue
        checkCuda(cudaGetLastError(), "launch queue_enq_kernel");                     // проверка запуска
        checkCuda(cudaDeviceSynchronize(), "sync queue_enq_kernel");                  // синхронизация
        checkCuda(cudaEventRecord(e2), "record queue enq end");                       // конец замера enqueue
        checkCuda(cudaEventSynchronize(e2), "sync queue enq end");                    // синхронизация события
        checkCuda(cudaEventElapsedTime(&queue_enq_ms, e1, e2), "elapsed queue enq");  // считаем время

        checkCuda(cudaEventRecord(e1), "record queue deq start");                     // старт замера dequeue
        queue_deq_kernel<<<blocks, threads>>>(d_queue, q_out, q_ok2, N);              // запуск dequeue
        checkCuda(cudaGetLastError(), "launch queue_deq_kernel");                     // проверка запуска
        checkCuda(cudaDeviceSynchronize(), "sync queue_deq_kernel");                  // синхронизация
        checkCuda(cudaEventRecord(e2), "record queue deq end");                       // конец замера dequeue
        checkCuda(cudaEventSynchronize(e2), "sync queue deq end");                    // синхронизация события
        checkCuda(cudaEventElapsedTime(&queue_deq_ms, e1, e2), "elapsed queue deq");  // считаем время

        bool stack_ok = check_unique_all(s_out, s_ok2, N);                            // проверяем корректность стека
        bool queue_ok = check_unique_all(q_out, q_ok2, N);                            // проверяем корректность очереди

        std::printf("%d,%.6f,%.6f,%d,%.6f,%.6f,%d\n",
                    N, stack_push_ms, stack_pop_ms, stack_ok ? 1 : 0,
                    queue_enq_ms, queue_deq_ms, queue_ok ? 1 : 0);                   // печатаем строку CSV

        checkCuda(cudaEventDestroy(e1), "event destroy e1");                          // удаляем событие
        checkCuda(cudaEventDestroy(e2), "event destroy e2");                          // удаляем событие

        checkCuda(cudaFree(q_out), "free q_out");                                     // освобождаем q_out
        checkCuda(cudaFree(q_ok2), "free q_ok2");                                     // освобождаем q_ok2
        checkCuda(cudaFree(q_ok1), "free q_ok1");                                     // освобождаем q_ok1
        checkCuda(cudaFree(s_out), "free s_out");                                     // освобождаем s_out
        checkCuda(cudaFree(s_ok2), "free s_ok2");                                     // освобождаем s_ok2
        checkCuda(cudaFree(s_ok1), "free s_ok1");                                     // освобождаем s_ok1
        checkCuda(cudaFree(queue_buf), "free queue_buf");                             // освобождаем буфер очереди
        checkCuda(cudaFree(stack_buf), "free stack_buf");                             // освобождаем буфер стека
        checkCuda(cudaFree(d_queue), "free d_queue");                                 // освобождаем очередь
        checkCuda(cudaFree(d_stack), "free d_stack");                                 // освобождаем стек
    }

    return 0;                                                                         // завершение
}


Writing stack_2_atomic.cu


In [9]:
!nvcc -O2 -std=c++17 stack_2_atomic.cu -o stack_2_atomic \
  -gencode arch=compute_75,code=sm_75
!./stack_2_atomic

stack_2_atomic.cu(142): warning #177-D: function "time_kernel" was declared but never referenced
  static float time_kernel(void (*launcher)(), cudaEvent_t a, cudaEvent_t b) {
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

N,stack_push_ms,stack_pop_ms,stack_ok,queue_enq_ms,queue_deq_ms,queue_ok
10000,0.096256,1.573472,1,0.042944,0.163840,1
100000,0.321536,0.020480,1,0.171008,0.241152,1
1000000,2.918240,1.212416,1,3.476928,1.292288,1


**Вывод**

Для небольшого размера массива (N = 10 000) операции с очередью выполняются быстрее, чем со стеком: как enqueue, так и dequeue занимают меньше времени по сравнению с push и pop. Это связано с более равномерным распределением операций между индексами head и tail.

При среднем размере (N = 100 000) время выполнения операций стека и очереди становится сопоставимым, при этом стек показывает более быстрый pop, а очередь — более стабильное время enqueue и dequeue.

Для большого объёма данных (N = 1 000 000) видно, что очередь начинает работать медленнее стека при добавлении элементов (enqueue > push). Это объясняется тем, что в очереди происходит конкуренция потоков сразу за две атомарные переменные (head и tail), тогда как в стеке используется только одна (top).

Таким образом, стек демонстрирует лучшую масштабируемость при больших размерах данных, тогда как очередь может быть быстрее на малых объёмах, но сильнее страдает от атомарных конфликтов при росте числа потоков.